# Calculating Refrigerant and Air Properties Using CoolProp Library

## Refrigerant Side Calculation

__Import CoolProp as CP for refrigerant propeties.__

In [1]:
import CoolProp.CoolProp as CP

__1. Select refrigerant, condenser inlet temperature, condenser inlet pressure, condenser outlet temperature, 
condenser outlet pressure.__

In [2]:
fluid = 'R410A'
T_inlet = 84 + 273.15                                                                                                             # Kelvin
P_inlet = 3.2 * 1e6                                                                                                               # Pa
T_outlet = 46.1 + 273.15                                                                                                          # Kelvin
P_outlet = 3.19 * 1e6                                                                                                             # Pa

__2. Calculate inlet enthalpy and outlet enthalpy using CoolProp's *PropsSI* function. The function's first argument is H, 
which indicates we are caculating enthalpy properties. Second and fourth arguments are T and P which indicates temperature and pressure
will be used to calculate the enthalpy. Finally in third, fifth and sixth argument temperature value, pressure value and refrigerant
type are provided.__

In [3]:
h_inlet = CP.PropsSI('H', 'T', T_inlet, 'P', P_inlet, fluid)                                                                      # J/kg
h_outlet = CP.PropsSI('H', 'T', T_outlet, 'P', P_outlet, fluid)                                                                   # J/kg

__3. Calculate the enthalpy difference.__

In [4]:
delta_h = (h_inlet - h_outlet) / 1000                                                                                             # kJ/Kg
print(f'Enthalpy difference: {delta_h} kJ/kg')

Enthalpy difference: 196.24982964457206 kJ/kg


__4. Select the target capacity for the condenser.__

In [5]:
capacity_target = 28.13                                                                                                           # kW

__5. Calculate the mass flow rate of the refrigerant for the target capacity from equation- Q = ṁ x Δh__

In [6]:
m_ref = (capacity_target / delta_h) * 3600                                                                                        # kg/h
print(f'Required mass flow rate of 8 TR AC: {m_ref} kg/h')

Required mass flow rate of 8 TR AC: 516.0157345532805 kg/h


## Air Side Calculation

__Import CoolProp.HumidAirProp as HA for humid air properties calculation.__

In [7]:
import CoolProp.HumidAirProp as HA

__1. Set room condition air temperature and relative humadity.__

In [8]:
air_temperature = 35 + 273.15                                                                                                     # Kelvin
relative_humidity = 0.45

__2. Calculate humdity ratio with HAPropsSI function. First argument is W which tells the function to return humdity ratio.
Second, fourth, and sixth argument are 'T', 'R', 'P' which indicates these parameters will be used to determine humidity ratio.
In third, fifth, seventh argument we will input temperature, relative humidity and atmospheric pressure value.__

In [9]:
humidity_ratio = HA.HAPropsSI('W', 'T', air_temperature, 'R', relative_humidity, 'P', 101325)
print(f'Humdity Ratio: {humidity_ratio}')

Humdity Ratio: 0.016021235695980525


__3. Next we calculate specific heat capacity at constant pressure for dry air.__

In [10]:
cp_dry_air = CP.PropsSI('C', 'T', air_temperature, 'P', 101325, 'Air') / 1000                                                     # kJ/kg . K
print(f"Specific Heat Capacity (Cp) of Dry Air at {air_temperature} K: {cp_dry_air} kJ/(kg·K)")

Specific Heat Capacity (Cp) of Dry Air at 308.15 K: 1.0066963047820279 kJ/(kg·K)


__4. Calculate specific heat capacity at constant pressure for water vapour at environmental condition__

In [11]:
cp_water_vapor = CP.PropsSI('C', 'T', air_temperature, 'P', 101325, 'Water') / 1000                                               # kJ/kg . K
print(f"Specific Heat Capacity (Cp) of Water Vapor at {air_temperature} K: {cp_water_vapor} kJ/(kg.K)")

Specific Heat Capacity (Cp) of Water Vapor at 308.15 K: 4.179258102223618 kJ/(kg.K)


__5. Specific heat capacity at constant pressure is calculated for moist air with the equation - 
Cp,moist air = Cp,dry air + ω . Cp, water vapor__

In [12]:
cp_moist_air = cp_dry_air + (humidity_ratio * cp_water_vapor)                                                                     # kJ/kg . K
print(f"Specific Heat Capacity (Cp) of Moist Air at {air_temperature} K: {cp_moist_air} kJ/(kg.K)")

Specific Heat Capacity (Cp) of Moist Air at 308.15 K: 1.0736531838720886 kJ/(kg.K)


__6. Set target outlet air temperature for condenser and compressor work.__

In [13]:
air_out_temperature = 47 + 273.15                                                                                                 # Kelvin
compressor_work = 7.02                                                                                                            # kW

__7. Calculate required mass flow of air using the equation - Q = ṁ x Cp x ΔT__

In [14]:
m_air = ((capacity_target + compressor_work) / (cp_moist_air * (air_out_temperature - air_temperature))) * 3600                   # kg/h
print(f'Required mass flow rate of air 8 TR AC: {m_air} kg/h')

Required mass flow rate of air 8 TR AC: 9821.607348072926 kg/h


__8. Calculate air density at the environment temperature and pressure using *PropsSI* function.__

In [15]:
air_density = CP.PropsSI('D', 'T', air_temperature, 'P', 101325, 'Air')                                                           # kg/m^3
print(f"Density of Air at {air_temperature} K: {air_density} kg/m³")

Density of Air at 308.15 K: 1.145787651724712 kg/m³


__9. Calculate the required volumetric flow rate using equation - q = ṁ / ρ__

In [16]:
volumetric_air_flow_rate = m_air / air_density                                                                                    # m^3/h
print(f'Required volumetric flow rate of air 8 TR AC: {volumetric_air_flow_rate} m^3/h')

Required volumetric flow rate of air 8 TR AC: 8571.92633668972 m^3/h
